In [1]:
import json
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt

In [4]:
# в словаре есть пропущенные значения age:,
def split_dict(dict_string):
    dict_string = dict_string[1:-1].split(',')
    f = lambda token: np.nan if len(token) < 2 or token[1] in ['','""'] else token[1]   
    return [f(i.split(':')) for i in dict_string]


def new_columns(df, columns):
    for col in columns:
        df[col] = 0

#### GETTING USERS

In [57]:
users = pd.read_csv('ThirtyMusic/entities/users.idomaar', sep='	', header=None)[[2,3]]

In [58]:
user_columns = list(ast.literal_eval(users.values[0][1]).keys())
user_columns

['lastfm_username',
 'gender',
 'age',
 'country',
 'playcount',
 'playlists',
 'subscribertype']

In [59]:
new_columns(users, user_columns)
users[user_columns] = np.array(users.apply(lambda x: split_dict(x[3]), axis=1).values.tolist())
users['ID'] = users[2]
users.drop([2,3], axis=1, inplace = True)

In [60]:
users.head()

,lastfm_username,gender,age,country,playcount,playlists,subscribertype,ID
0,"""000123""","""f""",24,"""US""",221012,2,"""base""",1116715959
1,"""000333""","""m""",39,"""CZ""",217535,9,"""base""",1163123792
2,"""00elen""","""f""",nan,nan,49733,2,"""base""",1184426573
3,"""00Eraser00""","""m""",32,"""DE""",168054,2,"""base""",1123157597
4,"""00fieldsy""","""m""",23,"""UK""",45700,2,"""base""",1171302116
...,...,...,...,...,...,...,...,...
45162,"""f1yb0y""","""m""",38,"""UK""",43728,3,"""base""",1126004101
45163,"""Cloud_Dog""","""m""",30,"""DE""",79743,2,"""base""",1169128340
45164,"""diesel64""","""m""",28,"""BE""",18130,2,"""base""",1136564558
45165,"""sjarel8""","""n""",nan,nan,34582,2,"""base""",1172631407


In [187]:
users.to_csv('users.csv', index=False)

#### GETTING ARTISTS

In [176]:
persons = pd.read_csv('ThirtyMusic/entities/persons.idomaar', sep='	', header=None)[[1, 3]]

In [179]:
persons['name'] = persons.apply(lambda x: x[3].split('"name":')[1][:-1], axis=1)

In [182]:
persons['ID'] = persons[1]
persons = persons[['ID', 'name']]

In [183]:
persons.head()

,ID,name
0,145148,"""Everything+Is+Illuminated"""
1,297899,"""Robin+O%27Brien"""
2,250429,"""Nicholas+Gunn++(2012)"""
3,32765,"""Aspasia+Stratigou"""
4,18689,"""Allison+Veltz"""


In [184]:
persons.to_csv('artists.csv', index=False)

#### GETTING TRACKS

In [100]:
track = pd.read_csv('ThirtyMusic/entities/tracks.idomaar', sep='	', header=None)[[1,3,4]]

In [164]:
# названия треков очень мусорные, поэтому, чтобы не грузить машину я не стал их брать
track_columns_features = ['duration', 'playcount']
new_columns(track, track_columns_features)

In [130]:
track['author_id'] = track.apply(lambda x: [person['id'] for person in ast.literal_eval(x[4].split('"albums"')[0]+'}')['artists']], axis=1)

meta_info = []
for x in track[3]:
    meta_info.append(split_dict(x)[:2])
track[track_columns_features] = meta_info

In [170]:
track['ID'] = track[1]
track = track[['ID', 'duration', 'author_id']]

In [186]:
track.head()

,ID,duration,name,author_id
0,0,-1,0,[0]
1,1,-1,0,[1]
2,2,-1,0,[2]
3,3,-1,0,[3]
4,4,-1,0,[4]


In [185]:
track.to_csv('tracks.csv', index=False)

#### GETTING PLAYLISTS

In [7]:
def extract_tracks(track_dict):
    track_dict = ast.literal_eval(track_dict)
    user_id = track_dict['subjects'][0]['id']
    ids = np.nan
    if track_dict['objects'] and track_dict['objects'][0]:
        ids = [track['id'] for track in track_dict['objects']]
    return user_id, ids

In [2]:
playlists = pd.read_csv('ThirtyMusic/entities/playlist.idomaar', sep='	', header=None)[[2,3,4]]

In [5]:
track_info = ['user_id', 'tracks_ids']
playlist_info = list(ast.literal_eval(playlists[3][0]).keys())
new_columns(playlists, track_info)
new_columns(playlists, playlist_info)

In [8]:
meta_info_track = []
for x in playlists[4]:
    meta_info_track.append(extract_tracks(x))
meta_info_playlist = []
for x in playlists[3]:
    current_info = []
    x = x.split('"ID":')[1].split(',', maxsplit=1)
    current_info.append(x[0])
    x = x[1].split(':', maxsplit=1)[1].split(',"numtracks":')
    current_info.append(x[0])
    x = x[1].split(',"duration":')
    current_info.append(x[0])
    current_info.append(x[1][:-1])
    meta_info_playlist.append(current_info)


In [9]:
playlists[track_info] = meta_info_track
playlists[playlist_info] = meta_info_playlist
playlists.dropna(subset=['tracks_ids'], inplace=True)

In [10]:
playlists['ID'] = playlists[2]
playlists = playlists[sum([track_info,playlist_info],[])]

In [11]:
playlists['numtracks'] = playlists['numtracks'].astype(int)

In [12]:
# уберем плейлисты в которых меньше трех треков
playlists = playlists[playlists.numtracks > 3]

In [13]:
playlists.head()

,user_id,tracks_ids,ID,Title,numtracks,duration
0,41504,"[3006631, 1885124, 2548942, 1350486, 3734368, ...",1216545588,"""my_favorites""",27,6522
1,41504,"[2807573, 2807606, 1119655, 1378148, 805788, 2...",1249326867,"""""",9,2650
2,44542,"[2236990, 3005440, 2941671, 3004987, 2834239, ...",1257766688,"""1989""",16,3645
3,44542,"[2338780, 1813232, 199079, 3295925, 1175085, 3...",1248079275,"""Fly me to the moon...""",15,4151
4,44542,"[1632714, 2933827, 1632721, 2533642, 2652489, ...",1175201268,"""The playlist who must not be named""",84,18414


In [14]:
playlists.to_csv('playlists.csv', index=False)

#### GETTING SESSION

In [397]:
sessions = pd.read_csv('ThirtyMusic/relations/sessions.idomaar', sep='	', header=None)

In [449]:
session_cols = ['numtracks', 'playtime', 'user_id', 'tracks', 'negative_samples']
new_columns(sessions, session_cols)

In [453]:
meta_info_session = []
for x in sessions[3]:
    x = x.split(' ')
    meta_info_session_buff = []
    meta_info_session_buff.append(ast.literal_eval(x[0])['numtracks'])
    meta_info_session_buff.append(ast.literal_eval(x[0])['playtime'])
    meta_info_session_buff.append(json.loads(x[1])['subjects'][0]['id'])
    song_ids = []
    negative_samples = []
    for song in json.loads(x[1])['objects']:
        if song['action'] == 'skip' or song['playtime'] <= 15:
            negative_samples.append(song['id'])
#       если песня играла меньше 15 секунд, то считаем что это негативный пример
#       если песню остановили, или скипнули тоже будем считать негативным примером
        elif song['action'] is None:
            negative_samples.append(song_ids.pop())
        elif song['playtime'] > 15:
            song_ids.append(song['id'])
    meta_info_session_buff.append(song_ids)
    meta_info_session_buff.append(negative_samples)
    meta_info_session.append(meta_info_session_buff)

In [454]:
sessions[session_cols] = meta_info_session

In [460]:
sessions = sessions[[1,2,'numtracks','playtime', 'user_id','tracks', 'negative_samples']]

# уберем сессии в которых не прослушали ни одну песню 
sessions = sessions[sessions['playtime'] != -1]

# возьмем сессии где прослушали более трех треков
sessions = sessions[sessions.numtracks > 3]

In [465]:
sessions.head()

,1,2,numtracks,playtime,user_id,tracks,negative_samples
0,287144,1390231051,23,4547,44361,"[4698874, 838286, 2588097, 2746740, 3873988, 8...",[4698881]
1,287145,1390241844,11,2907,44361,"[455834, 2460503, 1999451, 3257000, 2765459, 4...",[249947]
2,287146,1390303249,16,3191,44361,"[4698883, 1299602, 1907900, 1646559, 118991, 1...",[2897013]
3,287147,1390481828,5,1162,44361,"[2503252, 1329082, 1066876, 4678485]",[1876964]
5,287141,1421494124,32,7349,42773,"[485883, 3287727, 3287736, 3287725, 3287729, 3...",[1325417]


In [464]:
sessions.to_csv('sessions.csv', index = False)